# Geração de instâncias

In [1]:
import math
import json

In [2]:
# Salva um arquivo JSON com o dicionário fornecido
def imprimir_json(nome_arquivo, dados):
    with open(nome_arquivo, 'w', encoding='utf-8') as arquivo:
        json.dump(dados, arquivo, ensure_ascii = False, indent = 4)

In [3]:
# Número de vértices possíveis
V = [100, 150, 200, 250]
instancias = []

for v in V:
    coordenadas = []
    
    K = [0, int(v/2), v]
    
    with open('mo824_atividade2_coords','r') as coordsFile:
        for i in range(v):
            line = next(coordsFile)
            coordenadas.append([int(x) for x in line.split()])
            
    custos = {(i, j): [
        math.sqrt(sum((coordenadas[i][k]-coordenadas[j][k])**2 for k in range(2))),
        math.sqrt(sum((coordenadas[i][k+2]-coordenadas[j][k+2])**2 for k in range(2)))]
        for i in range(v) for j in range(v)}
    
    custosCopia = {str((i, j)): [
        math.sqrt(sum((coordenadas[i][k]-coordenadas[j][k])**2 for k in range(2))),
        math.sqrt(sum((coordenadas[i][k+2]-coordenadas[j][k+2])**2 for k in range(2)))]
        for i in range(v) for j in range(v)}
    
    for k in K:
        instancia = {}
        instancia["k"] = k
        instancia["custos"] = custos
        instancia["V"] = v
        instancias.append(instancia)
        
        instancia.update({"custos":custosCopia})
        imprimir_json(f'instancia-vertices-{v}-similaridade-{k}.json', instancia)

# k-TSP

In [4]:
import sys
import math
import random
from itertools import combinations
import gurobipy as gp
from gurobipy import GRB

In [5]:
# Callback - use lazy constraints to eliminate sub-tours
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                                if vals[i, j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < n:
            # add subtour elimination constr. for every pair of cities in tour
            model.cbLazy(gp.quicksum(model._vars[i, j]
                                     for i, j in combinations(tour, 2))
                         <= len(tour)-1)

In [6]:
# Given a tuplelist of edges, find the shortest subtour

def subtour(edges):
    unvisited = list(range(n))
    cycle = range(n+1)  # initial length has 1 more city
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(cycle) > len(thiscycle):
            cycle = thiscycle
    return cycle

In [7]:
n = 20

# Create n random points

random.seed(1)
points = [(random.randint(0, 100), random.randint(0, 100)) for i in range(n)]

# Dictionary of Euclidean distance between each pair of points

dist = {(i, j):
        math.sqrt(sum((points[i][k]-points[j][k])**2 for k in range(2)))
        for i in range(n) for j in range(i)}

m = gp.Model()

# Create variables

vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='e')
for i, j in vars.keys():
    vars[j, i] = vars[i, j]  # edge in opposite direction

# You could use Python looping constructs and m.addVar() to create
# these decision variables instead.  The following would be equivalent
# to the preceding m.addVars() call...
#
# vars = tupledict()
# for i,j in dist.keys():
#   vars[i,j] = m.addVar(obj=dist[i,j], vtype=GRB.BINARY,
#                        name='e[%d,%d]'%(i,j))


# Add degree-2 constraint

m.addConstrs(vars.sum(i, '*') == 2 for i in range(n))

# Using Python looping constructs, the preceding would be...
#
# for i in range(n):
#   m.addConstr(sum(vars[i,j] for j in range(n)) == 2)


# Optimize model

m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

vals = m.getAttr('x', vars)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

tour = subtour(selected)
assert len(tour) == n

print('')
print('Optimal tour: %s' % str(tour))
print('Optimal cost: %g' % m.objVal)
print('')

Set parameter Username
Academic license - for non-commercial use only - expires 2022-05-17
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 20 rows, 190 columns and 380 nonzeros
Model fingerprint: 0x32ae205a
Variable types: 0 continuous, 190 integer (190 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.00s
Presolved: 20 rows, 190 columns, 380 nonzeros
Variable types: 0 continuous, 190 integer (190 binary)

Root relaxation: objective 4.484281e+02, 31 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  448.42814    0    6          -  448.42814      -     -  